In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_deterministic_ops=true"
import jax, optax, json, argparse
from jax import numpy as jnp
from flax import nnx
from models import LeNet, ResNet
from fedflax import train, aggregate
from data import get_data
from functools import reduce, partial
from matplotlib import pyplot as plt
from utils import return_l2, angle_err, opt_create
from collections import defaultdict

## Setup
Based on command-line params

In [ ]:
# Data 
ds_train = get_data(beta=1., skew="feature", discrete=False)
ds_val = get_data(partition="val", beta=1., batch_size=32, skew="feature", discrete=False)
ds_test = get_data(partition="test", beta=1., batch_size=16, skew="feature", discrete=False)

# All various settings to find conditions in which the asymmetry helps
params_sweep:tuple[dict,...] = (
    # Baseline
    {},
    {"model_class":ResNet},

    # W-Asymmetry
    {"wasym":"densest", "kappa":5.},
    {"wasym":"densest", "kappa":1.},
    {"wasym":"densest", "kappa":.1},
    {"wasym":"densest", "kappa":1e-2},
    {"wasym":"densest", "kappa":0.},

    {"wasym":"random", "kappa":5.},
    {"wasym":"random", "kappa":1.},
    {"wasym":"random", "kappa":.1},
    {"wasym":"random", "kappa":1e-2},
    {"wasym":"random", "kappa":0.},

    {"model_class":ResNet, "wasym":"densest", "kappa":5.},
    {"model_class":ResNet, "wasym":"densest", "kappa":1.},
    {"model_class":ResNet, "wasym":"densest", "kappa":.1},
    {"model_class":ResNet, "wasym":"densest", "kappa":1e-2},
    {"model_class":ResNet, "wasym":"densest", "kappa":0.},

    {"model_class":ResNet, "wasym":"random", "kappa":5.},
    {"model_class":ResNet, "wasym":"random", "kappa":1.},
    {"model_class":ResNet, "wasym":"random", "kappa":.1},
    {"model_class":ResNet, "wasym":"random", "kappa":1e-2},
    {"model_class":ResNet, "wasym":"random", "kappa":0.},

    # SyRe
    {"sigma":1e-4},
    {"sigma":1e-3},
    {"sigma":5e-3},
    {"sigma":1e-2},
    {"sigma":5e-2},

    {"model_class":ResNet, "sigma":1e-4},
    {"model_class":ResNet, "sigma":1e-3},
    {"model_class":ResNet, "sigma":5e-3},
    {"model_class":ResNet, "sigma":1e-2},
    {"model_class":ResNet, "sigma":5e-2},

    {"sigma":1e-4, "weight_decay":2e-3},
    {"sigma":1e-3, "weight_decay":2e-3},
    {"sigma":5e-3, "weight_decay":2e-3},
    {"sigma":1e-2, "weight_decay":2e-3},
    {"sigma":5e-2, "weight_decay":2e-3},
    
    {"model_class":ResNet, "sigma":1e-4, "weight_decay":2e-3},
    {"model_class":ResNet, "sigma":1e-3, "weight_decay":2e-3},
    {"model_class":ResNet, "sigma":5e-3, "weight_decay":2e-3},
    {"model_class":ResNet, "sigma":1e-2, "weight_decay":2e-3},
    {"model_class":ResNet, "sigma":5e-2, "weight_decay":2e-3},

    # Bias ordering
    {"orderbias":True},
    {"model_class":ResNet,"orderbias":True},

    # Kernel normalization
    {"normweights":True},
    {"model_class":ResNet,"normweights":True}
)

## Check metrics
Available techniques are dimension expansion and W-Asymmetry, of which the latter is implemented here.

In [ ]:
def angle(updates):
    update_g = jax.tree.map(lambda updates: jnp.mean(updates, axis=0), updates)
    update_g = jnp.concatenate([jnp.ravel(x) for x in update_g])
    updates_flat = jnp.concatenate(jax.tree.map(lambda x: jnp.reshape(x, (4,-1)), updates), axis=1)
    for update in updates_flat:
        angle = jnp.degrees(jnp.arccos(optax.losses.cosine_similarity(update_g, update))).item()
    return angle

def l1(updates):
    update_g = jax.tree.map(lambda updates: jnp.mean(updates, axis=0), updates)
    update_g = jnp.concatenate([jnp.ravel(x) for x in update_g])
    updates_flat = jnp.concatenate(jax.tree.map(lambda x: jnp.reshape(x, (4,-1)), updates), axis=1)
    for update in updates_flat:
        l1 = jnp.abs(update_g-update).item()
    return l1

metrics = defaultdict(defaultdict)
for params in params_sweep:
    print(k:=f"{params=}")
    # Train
    opt_create = partial(opt_create, learning_rate=1e-3, weight_decay=params.pop("weight_decay", 1e-4))
    model = params.pop("model_class", LeNet)
    model = model(jax.random.key(42), **params, dim_out=2)
    updates, _ = train(model, opt_create, ds_train, ds_val, return_l2(0.), local_epochs="early", rounds=1, val_fn=angle_err, max_patience=5)
    # Aggregate
    model_g = aggregate(model, updates)
    # Average accuracy on local data
    vval_fn = nnx.jit(nnx.vmap(angle_err, in_axes=(None,0,0,0)))
    err_test = reduce(lambda e, batch: e + vval_fn(model_g,*batch), ds_test, 0.) / len(ds_test)
    err_val = reduce(lambda e, batch: e + vval_fn(model_g,*batch), ds_val, 0.) / len(ds_val)
    metrics[k]["test_err"] = err_test.squeeze().tolist()
    metrics[k]["val_err"] = err_val.squeeze().tolist()
    # Angle
    metrics[k]["angle"] = angle(jax.tree.leaves(updates))
    # L1 distance
    metrics[k]["l1"] = l1(jax.tree.leaves(updates))
    # Save
    json.dump(metrics, open(f"break/metrics.json", "w"))

## Visualize

In [ ]:
# Rendering style
plt.style.use("seaborn-v0_8-pastel")
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "font.sans-serif": ["Helvetica"],
    "text.latex.preamble": r"""
        \usepackage{amsmath, amssymb}
        \usepackage{mathptmx} 
    """
})

# Specify which setting to plot and to exclude
metrics = json.load(open(f"break/metrics.json", "r"))
keys = list(map(lambda x: "params="+str(x), params_sweep[1:2]+params_sweep[37:]))
err_test = jnp.asarray([metrics[k]["test_err"] for k in keys])
err_val = jnp.asarray([metrics[k]["val_err"] for k in keys])
angles = jnp.asarray([metrics[k]["angle"] for k in keys])
x_axis = [0., 1e-4, 1e-3, 5e-3, 1e-2, 5e-2] # [5., 1., .1, 1e-2, 0.]

# Scatter angles
fig, ax1 = plt.subplots(dpi=300);
ax1.scatter(x_axis, angles, color="C0", label="Client Drift", marker="D", s=20)
ax1.plot(x_axis, angles, color="C0", linewidth=1)
ax1.set_ylabel("Client Drift (°)")

# New axis for error boxplots
ax2 = ax1.twinx();
ax2.scatter(x_axis, err_test.mean(axis=1), color="C1", label="Error", marker="s", s=20)
ax2.plot(x_axis, err_test.mean(axis=1), color="C1", linewidth=1)
ax2.fill_between(x_axis, err_test.min(1), err_test.max(1), color="C1", alpha=0.5, step="mid", linewidth=0)
ax2.set_ylabel("Error")

# Final touches
ax1.set_xscale("log")
ax1.set_xlim(min(x_axis), max(x_axis))
ax1.set_xticklabels(ax1.get_xticks().round(4))
ax1.set_xlabel("Application of asymmetry parameter")
fig.legend()
fig.savefig(f"break/sigma_hiwd_resnet.png", bbox_inches="tight")